In [13]:
import pandas as pd
import numpy as np
from glob import glob
import time
from datetime import datetime
import csv
import ccxt

from data_provider import read_data_path, market_to_symbol
from TA_robots import rsi

market = 'ONT-BTC'
tf = '5m'

start_capital = 0.004
deposit_percent = 0.3

rsi_win = 12; rsi_buy = 38; rsi_sell = 76

robot_parameters = [rsi_win, rsi_buy, rsi_sell]
robot = rsi()
robot_name = robot.robot_name + '_' + market + '_' + tf

state_fn = 'Logs/' + robot_name + '.state'
state_file = glob(state_fn)
if state_file == []:
    market_position = 0
    amount  = 0
    buy_sum = 0
else:
    print('State file !!!')
    df_state = pd.read_csv(state_file[-1])    
    market_position = df_state['mp'][0]
    start_capital = df_state['capital'][0]
    amount = df_state['amount'][0]
    buy_sum = df_state['buy_sum'][0]


#Work 30
api_key = "7hOldNclzrzvMe3Xs8otllgdmsKLXIpctjP1eB009P9yd7EzIo5FqunaArrb0YFH"
api_secret = "0J5mmatEvePndM5JwpAIWPYo8ZCDSvxnqxsNPEYXCe35SSF1EDDOaiZQPR0Tenwn"

pd.set_option('precision', 9)
exchange = ccxt.binance({'apiKey': api_key,
                         'secret': api_secret})
commision = 0.00075

In [14]:
def write_trades(df_trades, trade):
    df_state = pd.DataFrame(columns=['mp', 'capital', 'amount'])
    df_state.loc[1] = [market_position, cur_capital, amount]
    df_state.to_csv(state_fn, index = False)
    
    print(trade[1])
    print('{d[0]} {d[1]:10s} {d[2]:.2f} {d[3]:.8f} {d[4]: .8f} {d[5]:.8f} {d[6]:.8f} {d[7]:.8f}'.format(d=trade))
    df_trades.loc[len(df_trades)] = trade
    fn =  'Logs/trades_' + robot_name + '.csv'
    df_trades.to_csv(fn , index = False)    
    

def create_market_buy_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_buy_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price

def create_market_sell_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_sell_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol,  order['timestamp'])
    price = trades[0]['price']
    return price
  

In [ ]:
data_path = read_data_path()
df_trades = pd.DataFrame(columns = ['Date', 'Trade type', 'amount', 'close price', 'buy price', 'cumProfit', 'curCapital', 'Deposit'])

symbol = market_to_symbol(market)
cur_capital = start_capital
cum_profit  = 0
deposit = 0
last_time = ''
tf_since_entry = 0

stop= False
print('Start trading:', str(datetime.now()), 'cur_capital:', cur_capital, 'amount:', amount)
while not stop:
    time.sleep (exchange.rateLimit / 1000) # time.sleep wants seconds
    #pd.read_csv не работает с сash file
    with open(data_path + '/Cash/' + market + '_' + tf + '.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        rl = list(reader)
    if rl != []:
        df = pd.DataFrame(rl[1:])
        df.columns = rl[0]
    df = df.dropna()
    if last_time > df.iloc[-1]['T']:
        print('time error', last_time, df.iloc[-1]['T'])
        continue
    elif last_time < df.iloc[-1]['T']:
        last_time = df.iloc[-1]['T']
        tf_since_entry += 1  
    else:
        continue
    
            
    tf_closes = df["C"].values.astype('float')
    
    price = tf_closes[-1]
    close_time = exchange.iso8601(int(df.iloc[-1]['T'])) 
    signals = robot.signals(tf_closes, *robot_parameters)
    
    #Long entry 
    if  market_position == 0 and signals[-1] == 1:

        amount = cur_capital / price * (1-commision) 
        amount = np.trunc(amount * 100)/100
        buy_price = create_market_buy_order(exchange, symbol, amount)
        
        buy_sum = buy_price * amount + cur_capital * commision
        cur_capital = 0;

        market_position = 1
        tf_since_entry = 0    
 
     
        write_trades(df_trades, [close_time, "entry", amount, price,  buy_price, cum_profit, cur_capital, deposit])
      #Long Exit Profit 
    if market_position == 1 and tf_since_entry > 1 and signals[-1] == -1:
 
        sell_price = create_market_sell_order(exchange, symbol, amount)

        profit = sell_price * amount - buy_sum
        cum_profit += profit
        deposit_sum = max(0, profit) * deposit_percent
        deposit += deposit_sum
        cur_capital = sell_price * amount - deposit_sum
        buy_sum = 0

        market_position = 0

        write_trades(df_trades, [close_time, "exit", amount, price, sell_price , cum_profit, cur_capital, deposit])
    #------------------Post processing---------------------------------------------------


    with open(state_fn, 'w') as state_file:
        state_file.write("time,mp,capital,amount,buy_sum\n") 
        state_file.write("%s,%s,%s,%s, %s" % (close_time, market_position, cur_capital, amount, buy_sum))

    df_stop = pd.read_csv('markets_in_trade.csv')
    stop=df_stop[(df_stop['robot'] == robot.robot_name) & (df_stop['market'] == market) & (df_stop['tf'] == tf)]['stop']
    if stop.shape[0] == 0: 
        print('В файле markets_in_trade.csv нет установок для данного робота')
        break
    else:
        stop = bool(stop.values[0])
    if stop:
        if market_position == 1: 
            sell_price = create_market_sell_order(exchange, symbol, amount)

            profit = sell_price * amount - buy_sum
            cum_profit += profit
            deposit_sum = max(0, profit) * deposit_percent
            deposit += deposit_sum
            cur_capital = sell_price * amount - deposit_sum

            market_position = 0

            write_trades(df_trades, [close_time, "stop", amount, price, sell_price , cum_profit, cur_capital, deposit])
        break

Start trading: 2018-08-25 19:02:03.871620 cur_capital: 0.004 amount: 12.39
time error 1535213100000 1535102700000
time error 1535214600000 1535104200000


In [18]:
last_time

'1535164500000'

In [20]:
df.iloc[-1]

T    None
O    None
H    None
L    None
C    None
V    None
Name: 500, dtype: object

In [22]:
df

T          O          H          L          C         V
0    1535014800000  0.0003207  0.0003208  0.0003202  0.0003208  29661.16
1    1535015100000  0.0003206  0.0003214    0.00032  0.0003209  33762.09
2    1535015400000  0.0003209  0.0003222  0.0003193   0.000321  29915.95
3    1535015700000  0.0003202  0.0003204  0.0003198  0.0003204  21292.37
4    1535016000000    0.00032  0.0003217    0.00032  0.0003202  24168.14
5    1535016300000  0.0003202  0.0003209   0.000318  0.0003181   40366.8
6    1535016600000   0.000318  0.0003207   0.000318  0.0003196  40000.27
7    1535016900000  0.0003196  0.0003209  0.0003193  0.0003205  36049.81
8    1535017200000  0.0003205  0.0003209  0.0003192  0.0003207  22897.13
9    1535017500000  0.0003202   0.000321  0.0003195  0.0003197  28421.16
10   1535017800000  0.0003194  0.0003198   0.000316   0.000317  76681.82
11   1535018100000  0.0003171  0.0003189  0.0003167  0.0003182   30702.1
12   1535018400000  0.0003185  0.0003208   0.000318  0.0003202  35232.02
13   1535018700000  0.0003202  0.0003202  0.0003172  0.0003181  31038.36
14   1535019000000   0.000318  0.0003187  0.0003174  0.0003177  26441.91
15   1535019300000  0.0003178  0.0003179   0.000317  0.0003175  25501.09
16   1535019600000  0.0003174  0.0003175  0.0003166  0.0003167  27381.39
17   1535019900000  0.0003167  0.0003172  0.0003162  0.0003165  28812.68
18   1535020200000  0.0003166  0.0003167  0.0003161  0.0003165  39811.97
19   1535020500000  0.0003167   0.000317  0.0003158  0.0003161  41102.36
20   1535020800000  0.0003162   0.000318  0.0003161  0.0003176  33202.61
21   1535021100000  0.0003175   0.000318  0.0003166  0.0003171  21236.69
22   1535021400000   0.000317  0.0003198   0.000317  0.0003192  27121.58
23   1535021700000  0.0003189  0.0003196   0.000318  0.0003186  29452.33
24   1535022000000  0.0003187  0.0003193  0.0003181   0.000319  24846.91
25   1535022300000  0.0003191  0.0003197  0.0003183  0.0003197   37418.0
26   1535022600000  0.0003194  0.0003212  0.0003194  0.0003204  47666.94
27   1535022900000  0.0003204  0.0003206  0.0003197  0.0003201   35431.6
28   1535023200000  0.0003199  0.0003206  0.0003187  0.0003193  28906.17
29   1535023500000  0.0003194  0.0003195  0.0003168  0.0003168  27621.69
..             ...        ...        ...        ...        ...       ...
471  1535156100000  0.0003229   0.000325  0.0003227   0.000325   8417.72
472  1535156400000  0.0003239  0.0003254  0.0003232  0.0003236  22201.29
473  1535156700000  0.0003236  0.0003245  0.0003229  0.0003235   4108.04
474  1535157000000  0.0003235  0.0003242  0.0003233  0.0003235   3982.13
475  1535157300000  0.0003235  0.0003246  0.0003226  0.0003241   9683.75
476  1535157600000  0.0003241  0.0003248  0.0003237  0.0003237   7545.46
477  1535157900000  0.0003237  0.0003246  0.0003236   0.000324   2569.29
478  1535158200000  0.0003237  0.0003249  0.0003234  0.0003239   8280.01
479  1535158500000  0.0003237  0.0003246  0.0003236  0.0003241   9296.29
480  1535158800000  0.0003241  0.0003248  0.0003237  0.0003243   8642.61
481  1535159100000  0.0003242  0.0003244  0.0003222  0.0003223   9962.96
482  1535159400000  0.0003223  0.0003228   0.000321  0.0003222   9778.48
483  1535159700000  0.0003217  0.0003231  0.0003211  0.0003217   7080.26
484  1535160000000  0.0003218  0.0003226  0.0003211  0.0003223   2516.32
485  1535160300000  0.0003223  0.0003224  0.0003212  0.0003222   2110.67
486  1535160600000   0.000322  0.0003225  0.0003215  0.0003218   2609.57
487  1535160900000  0.0003217  0.0003218  0.0003209  0.0003217   3715.04
488  1535161200000  0.0003215  0.0003222  0.0003212  0.0003222   7283.02
489  1535161500000  0.0003221  0.0003227  0.0003214  0.0003221  12837.76
490  1535161800000  0.0003217  0.0003223  0.0003209  0.0003223   3997.04
491  1535162100000  0.0003223  0.0003234  0.0003221  0.0003229  11930.12
492  1535162400000  0.0003229   0.000323  0.0003207  0.0003226  13774.08
493  1535162700000  0.0003226   0.000323  0.0003216  0.0003218   5541.93
49